In [ ]:
#Only use if sandbox is not installed
import os,sys
sys.path.append('../../../')
#import warnings
#warnings.filterwarnings("ignore")

## Initialize Sensor and Projector

In [ ]:
#save the paths to the calibration files 
from sandbox import _calibration_dir
_calibprojector = _calibration_dir + "my_projector_calibration.json"
_calibsensor = _calibration_dir + "my_sensor_calibration.json"

In [ ]:
#Import Sensor
from sandbox.sensor import Sensor
sensor = Sensor(calibsensor=_calibsensor, name="dummy",
               gauss_filter = False)

In [ ]:
# Import projector
from sandbox.projector import Projector
projector = Projector(calibprojector=_calibprojector)


## Initialize marker detection

In [ ]:
#Initialize the aruco detection
from sandbox.markers import MarkerDetection
aruco = MarkerDetection(sensor=sensor)

## Initialize main thread for live update

In [ ]:
from sandbox.main_thread import MainThread
main = MainThread(sensor=sensor, projector=projector, aruco=aruco)


In [ ]:
# Start the thread 
main.run()

### Control the features of the main thread

In [ ]:
main.widget_plot_module()

## Import the desired module

In [ ]:
# Import the modules to use and initialize them
from sandbox.modules.gimli import GeoelectricsModule

In [ ]:
geoelec = GeoelectricsModule()

## Add the module to the main thread

In [ ]:
main.add_module(name ='GeoelectricModule', module=geoelec)

In [ ]:
main.run()

## Steps to run the simulation

In [ ]:
main.stop()

### 1) create a mesh from the sandbox topography 

In [ ]:
#Get current frame
frame = sensor.get_frame()
extent = sensor.extent

In [ ]:
# Normalize the frame and get some real resistivity values 
frame2, extent2 = geoelec.scale_linear(frame, extent, vmin=5, vmax=1e5)
step = 7.5 # This is to generate a coarser mesh, for faster runtime simulation
mesh, data = geoelec.create_mesh(frame2, step)
print(mesh)

In [ ]:
# We can visualize the original fine mesh and the coarser mesh that will be
# used in the simulations, and a small comparison between both resolutions
fig = geoelec.show_mesh()

### 2) Position the 4 arucos and assign the id to the type of electrode

If no arucos available, you can manually set some dummy arucos in the model by doing:

In [ ]:
main.run()
aruco.set_aruco_position(dict_position={}) #be sure the dict is empty
# {id:[x_coordinate, y_coordinate]}
dict_position = {3: [50, 100],
                 2: [100, 100],
                 5: [150, 100],
                 6: [200, 100]}
aruco.set_aruco_position(dict_position=dict_position, frame=sensor.get_frame())
main.stop()
#Run cell again if arucos are still not visible

In [ ]:
# The id of the aruco will appear next to the place it is detected, or as a list doing:
df = geoelec.df_markers
assert len(df)==4
df
# The id is the index of the dataframe

In [ ]:
# 0 : electrode a
# 1 : electrode b
# 2 : electrode m
# 3 : electrode n
#Assign the aruco id to the electrode id
geoelec.set_id_aruco({3: 0,
                      6: 1,
                      2: 2,
                      5: 3})
assert geoelec.id is not None
geoelec.id

In [ ]:
geoelec.set_aruco_electrodes(df)
assert geoelec.electrode is not None
geoelec.electrode
# Remember, this will be the positions for the electrodes
# row 0 : electrode a
# row 1 : electrode b
# row 2 : electrode m
# row 3 : electrode n

### 3) create data container ERT

In [ ]:
# This will describe the type of measurement performed and create the geometric factors
import numpy as np
measurements = np.array([[0, 1, 2, 3],])
scheme_type = "abmn"  #Dipole-Dipole
    
scheme = geoelec.create_data_containerERT(measurements, scheme_type)
print(scheme)

### 4) calculate current flow based on the aruco positions

In [ ]:
sim, pot = geoelec.calculate_current_flow(time=True)

In [ ]:
fig = geoelec.show_streams(quiver=False)

### 5) perform a sensitivity analysis

In [ ]:
geoelec.calculate_sensitivity(time=True)

In [ ]:
fig = geoelec.show_sensitivity()

### To display the results in the thread of the sandbox use:

In [ ]:
main.run()

In [ ]:
geoelec.p_stream = True  # plot the stream lines
geoelec.view = "potential"  # to show the geoelectric field

In [ ]:
geoelec.p_stream = True  # plot the stream lines
geoelec.view = "sensitivity"  # to show the sensitivity analysis

In [ ]:
geoelec.p_stream = False  # plot the stream lines
geoelec.p_quiver = True  # plot the vector field of the geoelectric field
geoelec.view = "mesh"  # to show the geoelectric field

In [ ]:
# or is also possible to use the widgets

In [ ]:
geoelec.widgets_controller()

In [ ]:
main.stop()

## All the previous can be summarized using the widgets

In [ ]:
main.run()


In [ ]:
aruco.set_aruco_position(dict_position={}) #be sure the dict is empty
# {id:[x_coordinate, y_coordinate]}
dict_position = {0: [50, 150],
                 1: [100, 100],
                 2: [150, 100],
                 3: [200, 100]}
aruco.set_aruco_position(dict_position=dict_position, frame=sensor.get_frame())

In [ ]:
geoelec.show_widgets()

Note: You can use the same procedure in real time. This will work as long as the 4 arucos are detected. If you want to stop the simulation just remove or hide one of the arucos

In [ ]:
# stop thread of sandbox





main.stop()

In [ ]:
projector.ax.xaxis.get_ticks_position()

In [ ]:
projector.ax.